# [Scene Recognition with Bag-of-Words](https://www.cc.gatech.edu/~hays/compvision/proj4/)
For this project, you will need to report performance for three
combinations of features / classifiers. It is suggested you code them in
this order, as well:
1. Tiny image features and nearest neighbor classifier
2. Bag of sift features and nearest neighbor classifier
3. Bag of sift features and linear SVM classifier

The starter code is initialized to 'placeholder' just so that the starter
code does not crash when run unmodified and you can get a preview of how
results are presented.

## Setup

In [17]:
# Set up parameters, image paths and category list
%matplotlib notebook
%load_ext autoreload
%autoreload 2

import cv2
import numpy as np
import os.path as osp
import pickle
from random import shuffle
import matplotlib.pyplot as plt
from utils import *
import student_code as sc


# This is the list of categories / directories to use. The categories are
# somewhat sorted by similarity so that the confusion matrix looks more
# structured (indoor and then urban and then rural).
categories = ['Kitchen', 'Store', 'Bedroom', 'LivingRoom', 'Office', 'Industrial', 'Suburb',
              'InsideCity', 'TallBuilding', 'Street', 'Highway', 'OpenCountry', 'Coast',
              'Mountain', 'Forest'];
# This list of shortened category names is used later for visualization
abbr_categories = ['Kit', 'Sto', 'Bed', 'Liv', 'Off', 'Ind', 'Sub',
                   'Cty', 'Bld', 'St', 'HW', 'OC', 'Cst',
                   'Mnt', 'For'];

# Number of training examples per category to use. Max is 100. For
# simplicity, we assume this is the number of test cases per category, as
# well.
num_train_per_cat = 100

# This function returns lists containing the file path for each train
# and test image, as well as lists with the label of each train and
# test image. By default all four of these lists will have 1500 elements
# where each element is a string.
data_path = osp.join('..', 'data')
'''
steps = [5, 10, 20, 30, 40, 50]
C_value = [1, 2, 3, 4, 5, 10]
gamma_value = [0.1, 0.2, 0.3, 0.4, 0.5, 1]
vocab_steps = [5, 10, 20, 30]
bins = [3, 6, 9, 12]
'''

steps = [5, 10]
C_value = [1, 2, 3]
gamma_value = [0.5, 1, 1.5]
vocab_steps = [5, 10, 20]
bins = [3, 6, 9]

s = 5
C = 3
gamma = 1
vs = 20
vb = 9
    
hyper = [0, 0, 0, 0, 0, 0, 0];

#for vs in vocab_steps:
#    for vb in bins:
train_image_paths, test_image_paths, train_labels, test_labels = get_image_paths(data_path,
                                                                                 categories,
                                                                                 num_train_per_cat);

vocab_filename = 'vocab_50.pkl'

vocab_size = 50  # Larger values will work better (to a point) but be slower to compute
vocab = sc.build_vocabulary(train_image_paths, vocab_size, vs, vb)
with open(vocab_filename, 'wb') as f:
    pickle.dump(vocab, f)

train_image_feats = sc.get_bags_of_sifts(train_image_paths, vocab_filename, s)
test_image_feats = sc.get_bags_of_sifts(test_image_paths, vocab_filename, s)

print(s, C, gamma, vs, vb)

predicted_categories = sc.nearest_neighbor_classify(train_image_feats, train_labels, test_image_feats)
cat2idx = {cat: idx for idx, cat in enumerate(categories)}
y_true = [cat2idx[cat] for cat in test_labels]
y_pred = [cat2idx[cat] for cat in predicted_categories]
cm = confusion_matrix(y_true, y_pred)
cm = cm.astype(np.float) / cm.sum(axis=1)[:, np.newaxis]
acc1 = np.mean(np.diag(cm))
sd = np.std(np.diag(cm))
print(acc1, sd, 'NN')

predicted_categories = sc.svm_classify(train_image_feats, train_labels, test_image_feats, 'linear', gamma, C)
cat2idx = {cat: idx for idx, cat in enumerate(categories)}
y_true = [cat2idx[cat] for cat in test_labels]
y_pred = [cat2idx[cat] for cat in predicted_categories]
cm = confusion_matrix(y_true, y_pred)
cm = cm.astype(np.float) / cm.sum(axis=1)[:, np.newaxis]
acc2 = np.mean(np.diag(cm))
sd = np.std(np.diag(cm))
print(acc2, sd, 'SVM linear')

if acc1 >= hyper[5] and acc2 >= hyper[6]:
    hyper = [s,C,gamma,vs,vb, acc1, acc2]

print(hyper)                   

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
5 2 1 10 6
0.5406666666666666 0.18419072241082668 NN


/Users/zhurikhinaa/miniconda2/envs/cs6476p4/lib/python3.5/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


0.6206666666666667 0.24447131165480795 SVM linear
[5, 2, 1, 10, 6, 0.5406666666666666, 0.6206666666666667]


/Users/zhurikhinaa/miniconda2/envs/cs6476p4/lib/python3.5/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
